In [1]:
import pandas as pd
import altair as alt

In [2]:
cols = ["IdLandkreis", "Meldedatum", "AnzahlFall", "NeuerFall"]
covid = pd.read_csv('https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.csv', usecols=cols, parse_dates=['Meldedatum'])
einwohner = pd.read_csv('../Einwohnerzahlen.csv')

In [171]:
covid.Meldedatum = pd.DatetimeIndex(covid.Meldedatum.dt.date)
covid.tail()

,AnzahlFall,Meldedatum,IdLandkreis,NeuerFall
1799152,3,2020-12-07,16077,0
1799153,3,2020-12-08,16077,0
1799154,4,2020-12-09,16077,0
1799155,1,2020-12-09,16077,0
1799156,1,2020-12-10,16077,0


In [172]:
key_data = covid[covid.NeuerFall != -1].groupby(['IdLandkreis', 'Meldedatum']).sum()
key_data

AnzahlFall  NeuerFall
IdLandkreis Meldedatum                       
1001        2020-03-14           4          0
            2020-03-18           2          0
            2020-03-19           4          0
            2020-03-20           2          0
            2020-03-21           1          0
...                            ...        ...
16077       2021-04-24          22          0
            2021-04-25          12          0
            2021-04-26          21          0
            2021-04-27          63          8
            2021-04-28           1          1

[128582 rows x 2 columns]

In [173]:
idx = pd.DatetimeIndex(pd.date_range(start=covid.Meldedatum.min(), end=covid.Meldedatum.max(), freq='1D'))
multi_idx = pd.MultiIndex.from_product([key_data.index.levels[0], idx], names=["IdLandkreis", "Meldedatum"])

key_data = key_data.reindex(multi_idx, fill_value=0)
key_data

AnzahlFall  NeuerFall
IdLandkreis Meldedatum                       
1001        2020-01-07           0          0
            2020-01-08           0          0
            2020-01-09           0          0
            2020-01-10           0          0
            2020-01-11           0          0
...                            ...        ...
16077       2021-04-24          22          0
            2021-04-25          12          0
            2021-04-26          21          0
            2021-04-27          63          8
            2021-04-28           1          1

[196936 rows x 2 columns]

In [174]:
cases_7d = []
for group, data in key_data.groupby(level=0):
    cases_7d.extend(data.reset_index().set_index('Meldedatum').rolling(window="7D").sum()['AnzahlFall'].to_list())
key_data['AnzahlFall7T'] = cases_7d

In [175]:
key_data.reset_index(inplace=True)

In [176]:
key_data = key_data.merge(einwohner, left_on="IdLandkreis", right_on="AdmUnitId")
key_data['Inz7T'] = key_data.AnzahlFall7T / key_data.EWZ * 100000

In [177]:
alt.Chart(key_data[key_data.reset_index().IdLandkreis.isin([12063, 12064, 3251])]).mark_line().encode(
    x='Meldedatum',
    y=alt.Y('Inz7T'),
    color="IdLandkreis:N",
).interactive(True).properties(width=800)

alt.Chart(...)

In [215]:
key_data[key_data.IdLandkreis == 5111].tail(10)

,IdLandkreis,Meldedatum,AnzahlFall,NeuerFall,AnzahlFall7T,AdmUnitId,GEN,BEZ,EWZ,BL_ID,BL,EWZ_BL,Inz7T,5T,3T
30582,5111,2021-04-19,65,0,945.0,5111,Düsseldorf,Kreisfreie Stadt,621877,5,Nordrhein-Westfalen,17947221,151.959310,152.120114,152.120114
30583,5111,2021-04-20,227,0,963.0,5111,Düsseldorf,Kreisfreie Stadt,621877,5,Nordrhein-Westfalen,17947221,154.853773,154.853773,154.853773
30584,5111,2021-04-21,156,0,981.0,5111,Düsseldorf,Kreisfreie Stadt,621877,5,Nordrhein-Westfalen,17947221,157.748236,157.748236,157.748236
30585,5111,2021-04-22,192,0,1019.0,5111,Düsseldorf,Kreisfreie Stadt,621877,5,Nordrhein-Westfalen,17947221,163.858769,163.858769,163.858769
30586,5111,2021-04-23,145,0,1028.0,5111,Düsseldorf,Kreisfreie Stadt,621877,5,Nordrhein-Westfalen,17947221,165.306001,165.306001,165.306001
30587,5111,2021-04-24,112,0,979.0,5111,Düsseldorf,Kreisfreie Stadt,621877,5,Nordrhein-Westfalen,17947221,157.426629,165.306001,165.306001
30588,5111,2021-04-25,118,0,1015.0,5111,Düsseldorf,Kreisfreie Stadt,621877,5,Nordrhein-Westfalen,17947221,163.215555,165.306001,165.306001
30589,5111,2021-04-26,67,0,1017.0,5111,Düsseldorf,Kreisfreie Stadt,621877,5,Nordrhein-Westfalen,17947221,163.537162,165.306001,163.537162
30590,5111,2021-04-27,163,36,953.0,5111,Düsseldorf,Kreisfreie Stadt,621877,5,Nordrhein-Westfalen,17947221,153.245738,165.306001,163.537162
30591,5111,2021-04-28,140,43,937.0,5111,Düsseldorf,Kreisfreie Stadt,621877,5,Nordrhein-Westfalen,17947221,150.672882,163.537162,163.537162


In [214]:
foo = key_data[key_data.Meldedatum > "2021-04-25"].groupby('IdLandkreis')['Inz7T'].agg(["min", "max"])
foo[(foo['min'] > 150) & (foo['max'] < 165)]

,min,max
IdLandkreis,,
5111,150.672882,163.537162
5766,153.662874,155.677181
6438,154.294531,164.131159
9163,155.780397,160.501015


In [216]:
einwohner[einwohner.AdmUnitId == 5111]

,AdmUnitId,GEN,BEZ,EWZ,BL_ID,BL,EWZ_BL
63,5111,Düsseldorf,Kreisfreie Stadt,621877,5,Nordrhein-Westfalen,17947221
